In [2]:
import numpy as np
import pandas as pd
import io
import regex as re
from helper import *

In [3]:
# Start with data files 1 - CombineStats
combineStats = []
filenums = ["09", "10", "11", "12", "13", "14", "15", "16"]
# Load the data
for i in filenums:
    year = 2000+int(i)
    data = pd.read_csv("DraftData/CombineStats/CombineStats_20{}.csv".format(i))
    data["PLAYER"] = formatNames(data["PLAYER"])
    data.replace("-", np.nan, inplace=True)
    data["year"] = year
    combineStats.append(data)
    
combineStats = pd.concat(combineStats)

In [4]:
# Now get 2 - CombineStats2
combineStats2 = []
filenums = ["10", "11", "12", "13", "14", "15"]
for i in filenums:
    year = 2000+int(i)
    data = pd.read_csv("DraftData/CombineStats2/CombineStats2_20{}.csv".format(i))
    data["PLAYER"] = formatNames(data["PLAYER"])
    data.replace("-", np.nan, inplace=True)
    combineStats2.append(data)
combineStats2 = pd.concat(combineStats2)

In [5]:
# Now get 3 - PlayerData LOTS HERE - GOING TO GET THE MOST RECENT YEAR OF DATA !!!
playerData = pd.read_csv("DraftData/PlayerData4.csv")
print("KEN", len(playerData))
playerData.drop_duplicates(subset=["Name"], inplace=True, keep="first")
#playerData.dropna(subset=["PerGame"], inplace=True)
print("KEN3", len(playerData))
playerData.reset_index(inplace=True, drop=True)
playerData["Name"] = formatNames(playerData["Name"])
playerData["Position"] = [playerData["Position"].iloc[i][15:].replace("\n", "").replace(" ", "") for i in range(len(playerData))]

# Columns that will be added from per game stats
playerData["collegeYear"] = str(np.nan)
playerData["gamesPlayed"] = np.nan
playerData["minutes"] = np.nan
playerData["FT%"] = np.nan
playerData["3P%"] = np.nan
playerData["SOS"] = np.nan

# Columns that will be added from advanced stats
advanced_columns = ["PER", "eFG%", "ORB%", "DRB%", "AST%", "TOV%", "STL%", "BLK%", "USG%", "OWS", "DWS"]
for col in advanced_columns:
    playerData[col] = np.nan
    
# Columns total will be added from total stats
total_columns = ["FTA", "FGA", "MP", "3PA", "PTS", "FGA", "PF", "TOV", "AST"]
for col in total_columns:
    playerData[col] = np.nan
# Calcualted total columns
playerData["MP_per_PF"] = np.nan
playerData["FTA_per_FGA"] = np.nan
playerData["MP_per_3PA"] = np.nan
playerData["PTS_per_FGA"] = np.nan
playerData['AST_per_TOV'] = np.nan
playerData['PPG'] = np.nan
playerData['PPM'] = np.nan

# Posession data
playerData["ORtg"] = np.nan
playerData["DRtg"] = np.nan

# Awards coumt
playerData["awardCount"] = 0

for i in range(len(playerData)):
    # PER GAME DATA
    name = playerData["Name"].iloc[i]

    perGame = playerData["PerGame"].iloc[i][80:]
    perGame = pd.read_csv(io.StringIO(perGame), lineterminator='\n')
    # Grab their most recent year
    perGame = perGame.iloc[perGame[perGame["Season"]=="Career"].index[0]-1, :]
    
    # Check that the guy is the right one
    if int(perGame["Season"][:4]) < 2009:
        print(perGame["Season"][:4], name)
    
    # Fill in the data
    playerData["collegeYear"].iloc[i] = perGame["Season"]
    playerData["gamesPlayed"].iloc[i] = perGame["G"]
    playerData["minutes"].iloc[i] = perGame["MP"]
    playerData["FT%"].iloc[i] = perGame["FT%"]
    playerData["3P%"].iloc[i] = perGame["3P%"]
    playerData["SOS"].iloc[i] = perGame["SOS"]
    
    if type(playerData["Advanced"].iloc[i]) == float:
        print(playerData.iloc[i]["Advanced"])
    # Now get the advanced stats
    advanced = playerData["Advanced"].iloc[i][80:]
    advanced = pd.read_csv(io.StringIO(advanced), lineterminator='\n')
    advanced = advanced.iloc[advanced[advanced["Season"]=="Career"].index[0]-1, :]
    
    # Fill in advanced data
    for col in advanced_columns:
        if col in advanced.index:
            playerData[col].iloc[i] = advanced[col]
            
    # Now get the totals
    totals = playerData["Totals"].iloc[i][80:]
    totals = pd.read_csv(io.StringIO(totals), lineterminator='\n')
    totals = totals.iloc[totals[totals["Season"]=="Career"].index[0]-1, :]
    # Fill in total data
    for col in total_columns:
        if col in totals.index:
            playerData[col].iloc[i] = totals[col]
        else:
            print(i, col)
    # Calcualte the derrive metrics for totals stats
    playerData["MP_per_PF"].iloc[i] = totals["MP"]/totals["PF"] if totals["PF"]!=0 else np.nan # SHOULD BE FIXED
    playerData["FTA_per_FGA"].iloc[i] = totals["FTA"]/totals["FGA"] # ASSUMED NOT ZERO
    playerData["MP_per_3PA"].iloc[i] = totals["MP"]/totals["3PA"] if totals["3PA"]!=0 else np.nan # SHOULD BE FIXED
    playerData["PTS_per_FGA"].iloc[i] = totals["PTS"]/totals["FGA"]
    playerData['AST_per_TOV'].iloc[i] = totals["AST"]/totals["TOV"]
    playerData['PPG'].iloc[i] = totals["PTS"]/totals["G"]
    playerData['PPM'].iloc[i] = totals['PTS']/totals["MP"]
    
    
    # Get offensive and defensive ratings
    per100 = playerData["Per100Pos"].iloc[i]
    if type(per100) == str:
        per100 = per100[80:]
        per100 = pd.read_csv(io.StringIO(per100), lineterminator="\n")
        per100 = per100.iloc[per100[per100["Season"]=="Career"].index[0]-1, :]
        playerData['ORtg'].iloc[i] = per100["ORtg"]
        playerData['DRtg'].iloc[i] = per100["DRtg"]
    
    # Now count their awards
    awards = playerData["Awards"].iloc[i]
    if type(awards)==str:
        playerData["awardCount"].iloc[i] = awards.count("20")


KEN 480
KEN3 353


/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [5]:
# Get the RSCI data
colNames = {"center":"RSCI", "left":"name", "left5":"collegeTeam", 
            "right8":"mock1", "right9":"mock2", "right10":"mock3", 
            "right11":"mock4", "right12":"mock5", "right13":"mock6"}
RSCI_cols= ["RSCI", "name", "collegeTeam", "mock1", "mock2", "mock3", "mock4", "mock5", "mock6"]
RSCI = []
RSCIs = ["06", "07", "08", "09", "10", "11", "12", "13", "14", "15"]
# Load the dat a
for i in RSCIs:
    data = pd.read_csv("DraftData/RSCI/RSCI_Mock_Data_20{}.csv".format(i))
    data.dropna(subset=["left"], inplace=True)
    data["left"] = [name.replace("(college)", "") for name in data["left"]]
    
    # Reformat column names
    data.rename(columns=colNames, inplace=True)
    
    # Drop some columns
    data = data[RSCI_cols]
    data["name"] = formatNames(data["name"])
    RSCI.append(data)
    
RSCI = pd.concat(RSCI).reset_index()

In [6]:
# Get the draft place data ... 
draftResults = []
fileNums = [str(i) for i in range(10, 18)]
for i in range(10, 18):
    data = pd.read_csv("DraftData/DraftResults/DraftResults_20{}.csv".format(str(i)))
    draftResults.append(data[["PLAYER", "TEAM", "AFFILIATION", "YEAR", "OVERALL_PICK"]])
draftResults = pd.concat(draftResults)
draftResults["PLAYER"] = formatNames(draftResults["PLAYER"])

In [7]:
# Get age data from win share dataset
WS_data = []

num_files = 8
for i in range(num_files):
    data = pd.read_csv("TargetData/BRData/AdvancedPlayerStats_201{}.csv".format(i))
    data = data[["Player", "Age"]]
    data["Player"] = formatNames(data["Player"])
    data["Year"] = 2010+i
    WS_data.append(data)
    
# Turn this into a single dataframe
WS_data = pd.concat(WS_data)

In [13]:
# COMBINE EVERYTHING !!!!!!
UNPLAYED_PLAYERS = []

final = playerData[["Name", "Position", "collegeYear", "gamesPlayed", "minutes", 'FT%', '3P%', 'SOS', 
                    'PER', 'eFG%', 'ORB%', 'DRB%', 'AST%', 'TOV%','STL%', 'BLK%', 'USG%', 'OWS', 'DWS', 
                    'FTA', 'FGA', 'MP', '3PA', 'PTS', 'PF', 'MP_per_PF', 'FTA_per_FGA', 'MP_per_3PA', 
                    'PPG', 'PPM', 'AST_per_TOV', 
                    'PTS_per_FGA', "ORtg", "DRtg", 'awardCount']].copy()
RSCI_cols= ["RSCI", "collegeTeam", "mock1", "mock2", "mock3", "mock4", "mock5", "mock6"]
combineCols = ['AGILITY', 'SHUTTLE_RUN', 'THREE_QUARTER_SPRINT', 'STANDING_VERITCAL', 'MAX_VERTICAL', 
               'BENCH_PRESS']
combineCols2 = ['BODY_FAT', 'HAND_LENGTH', 'HAND_WIDTH', 'HEIGHT', 'HEIGHT_W_SHOES', 'REACH', 'WEIGHT', 
                'WINGSPAN']
colsToAdd = ["draftYear", "age"] + RSCI_cols + combineCols + combineCols2 + ["overallPick", "NBAteam"]
for col in colsToAdd:
    final[col] = np.nan

for i in range(len(final)):
    name = final["Name"].iloc[i]
    
    draftYear = int("20"+final["collegeYear"].iloc[i][-2:])
    final["draftYear"].iloc[i] = draftYear
    
    # Get rookie age
    playerRows = WS_data.loc[WS_data["Player"]==name]
    if len(playerRows)==0:
        UNPLAYED_PLAYERS.append(name)
        continue
        
    age = playerRows.loc[playerRows["Year"]==min(playerRows["Year"])]["Age"].iloc[-1]
    final["age"].iloc[i] = age
    
    # Add RSCI data
    RSCI_entry = RSCI.loc[RSCI["name"]==name]
    if len(RSCI_entry)!=0:
        for col in RSCI_cols:
            final[col].iloc[i] = RSCI_entry[col].iloc[0]
    else:
        pass
            
    # Add combine stats
    combineStats_entry = combineStats.loc[combineStats["PLAYER"]==name]
    if len(combineStats_entry)!=0:
        for col in combineCols:
            final[col].iloc[i] = combineStats_entry[col].iloc[0]
            
    # Add combine stats 2 
    combineStats2_entry = combineStats2.loc[combineStats2["PLAYER"]==name]
    if len(combineStats2_entry)!=0:
        for col in combineCols2:
            final[col].iloc[i] = combineStats2_entry[col].iloc[0]
            
    draftResults_entry = draftResults.loc[draftResults["PLAYER"]==name]
    if len(draftResults_entry)>0:
        final["overallPick"].iloc[i] = draftResults_entry["OVERALL_PICK"].iloc[0]
        final["draftYear"].iloc[i] = draftResults_entry["YEAR"].iloc[0]
        final["NBAteam"].iloc[i] = draftResults_entry["TEAM"].iloc[0]
    
# final = final[~final['Name'].isin(UNPLAYED_PLAYERS)]
final.to_pickle("finalDraft.df")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
len(playerData)

353

In [15]:
len(final)

353

In [20]:
for i in range(len(final.isna().sum())):
    print(final.columns[i], final.isna().sum()[i])

Name 0
Position 0
collegeYear 0
gamesPlayed 0
minutes 0
FT% 0
3P% 38
SOS 0
PER 54
eFG% 0
ORB% 2
DRB% 2
AST% 2
TOV% 2
STL% 2
BLK% 2
USG% 2
OWS 2
DWS 2
FTA 0
FGA 0
MP 0
3PA 0
PTS 0
PF 0
MP_per_PF 0
FTA_per_FGA 0
MP_per_3PA 38
PPG 0
PPM 0
AST_per_TOV 0
PTS_per_FGA 0
ORtg 143
DRtg 143
awardCount 0
draftYear 0
age 27
RSCI 154
collegeTeam 155
mock1 173
mock2 168
mock3 167
mock4 161
mock5 165
mock6 184
AGILITY 141
SHUTTLE_RUN 253
THREE_QUARTER_SPRINT 139
STANDING_VERITCAL 136
MAX_VERTICAL 136
BENCH_PRESS 197
BODY_FAT 131
HAND_LENGTH 131
HAND_WIDTH 131
HEIGHT 132
HEIGHT_W_SHOES 132
REACH 132
WEIGHT 132
WINGSPAN 131
overallPick 27
NBAteam 27


In [17]:
draftResults

,PLAYER,TEAM,AFFILIATION,YEAR,OVERALL_PICK
0,johnwall,Washington Wizards,Kentucky,2010,1
1,evanturner,Philadelphia 76ers,Ohio State,2010,2
2,derrickfavors,New Jersey Nets,Georgia Tech,2010,3
3,wesleyjohnson,Minnesota Timberwolves,Syracuse,2010,4
4,demarcuscousins,Sacramento Kings,Kentucky,2010,5
...,...,...,...,...,...
55,jabaribird,Boston Celtics,California,2017,56
56,aleksandarvezenkov,Brooklyn Nets,FC Barcelona Basquet (Spain),2017,57
57,ognenjaramaz,New York Knicks,KK Mega Leks (Serbia),2017,58
58,jaronblossomgame,San Antonio Spurs,Clemson,2017,59
